1. requests를 이용 뉴스 사이트 가져오기 (https://news.nate.com/, https://media.daum.net/) <br>
3. 각 카테고리 (예) 정치/경제/사회/문화/세계/IT) <br>
4. 카테고리별 이름 추출 <br>
5. 각 뉴스로 이동 <br>
6. 각 뉴스의 본문 카테고리 찾기 <br>
7. 본문의 텍스트만 추출하여 저장 <br
File Name : 카테고리-aid.text ex) 정치-20190510190906476.txt

In [1]:
import requests
from bs4 import BeautifulSoup
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36'}

In [2]:
def getDownload(url, param = None, retries = 3):
    resp = None
    try:
        resp = requests.get(url, params = param, headers = header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retries > 0:
            print('Retries : {0}'.format(retries))
            return getDownload(url, param, retries -1)
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.request.headers)
            
    return resp

# 다음 뉴스기사

In [3]:
url = "https://media.daum.net/"
html = getDownload(url)
dom = BeautifulSoup(html.text, "lxml")

In [68]:
import re

# 정규표현식
def clean_text(text):
                cleaned_text = re.sub('[a-zA-Z]', '', text)
                cleaned_text = re.sub('[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]', '', cleaned_text)
                cleaned_text = re.sub('\n', '', cleaned_text)
                return cleaned_text
            

In [71]:
# 홈에서 뉴스 카테고리 선택
for _ in dom.select(".gnb_comm > li > a")[1:6]:
#     print(_.text) # 카테고리명 출력
    suburl = requests.compat.urljoin(url, _["href"])
    category = _.text
    
    html = getDownload(suburl)
    newdom = BeautifulSoup(html.text, "lxml")
            
    for _ in newdom.select("strong.tit_thumb > a.link_txt"):
#         print(_.text.strip())
#         print(_["href"])
        
        subsuburl = requests.compat.urljoin(suburl, _["href"])
        html = getDownload(subsuburl)
        newdom2 = BeautifulSoup(html.text, "lxml")
        
        #파일 저장할 경로
        path = "C:\\Users\\Yang Saewon\\Desktop\\project\\github\\natural_language_processing\\data\\"
        filename = _["href"].split('/')[-1] 
        
        
        # 해당 뉴스 본문 가져오기
        for _ in newdom2.select("div.news_view"):
            news_content_text = _.text.strip()
#             print(news_content_text)
            
            news_content_text = clean_text(news_content_text)

            
            with open(path + category + '_' + filename + ".txt", "w", encoding='utf-8') as f:
                f.write(news_content_text)
        
        
    

# 네이트 뉴스기사

In [79]:
url = "https://news.nate.com/"
html = getDownload(url)
dom = BeautifulSoup(html.text, "lxml")

In [ ]:
for _ in dom.select("[class^='news0'] > a")[2:7]: 
#     print(_.text) # 카테고리명 출력
    category = _.text
    
    suburl = requests.compat.urljoin(url, _["href"])

    html = getDownload(suburl)
    newdom = BeautifulSoup(html.text, "lxml")
    
    for _ in newdom.select("ul.mduList1 > li > a"):
        
#         print(_.text.strip())
#         print(_["href"])
                
        subsuburl = requests.compat.urljoin(suburl, _["href"])
        html = getDownload(subsuburl)
        newdom2 = BeautifulSoup(html.text, "lxml")
        
        
        #파일 저장할 경로
        path = "C:\\Users\\Yang Saewon\\Desktop\\project\\github\\natural_language_processing\\data\\"
        filename = _["href"].split('/')[-1][4:13]
        
        
        # 해당 뉴스 본문 가져오기
        for _ in newdom2.find_all('div',{'id':'realArtcContents'}):
            
            news_content_text = _.text.strip()            
            news_content_text = clean_text(news_content_text)

            with open(path + category + '_' + filename + ".txt", "w", encoding='utf-8') as f:
                f.write(news_content_text)

    

![123](https://user-images.githubusercontent.com/40786348/57563117-ec865b80-73d4-11e9-980e-ba005fdd156a.PNG)